In [17]:
import igl
#import meshplot as mp
#import vedo as vd
import polyscope as ps
import numpy as np
import os 
import time 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.HyperbolicLC import HyperbolicLC
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

#vd.settings.default_backend = 'k3d'

ModuleNotFoundError: No module named 'cmat'

## Initial Line Congruence test

### Visualization setup

In [ ]:
dir_path = os.getcwd()

data_path = dir_path+"/approximation/data/"

mv, mf = igl.read_triangle_mesh(os.path.join(data_path,"centers.obj" ))
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,"test_remeshed.obj" ))
# Get vertex normals for test mesh
tn = igl.per_vertex_normals(tv, tf)

signs = np.sign(np.sum(tn * ([0,0,1]), axis=1))

# Fix normal directions
tn = tn * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Create dual mesh
mesh = Mesh()

mesh.make_mesh(tv,tf)

# Get inner vertices
inner_vertices = mesh.inner_vertices()

# Dual topology including outer faces
dual = mesh.dual_top()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]


# dual_faces = dualmesh.faces()
# # Draw vertex normals
# # vd.show(vd.Points(tv, c='red'), vd.Arrows(tv, tv + tn * 0.1, c='blue'))

# # Draw circles
# #circles = [ vd.Line(vd.Circle(ct, rt, c='k').orientation(nt), closed=True, lw=1.2, c='black') for ct, rt, nt in zip(ct, rt, nt) ]

# Create test mesh and center mesh 
mesh =  vd.Mesh((tv, tf), alpha = 0.8)
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

# Create hexagonal mesh
hexmesh = vd.Mesh((h_pts, dual_faces), alpha = 0.5, c=[0.2, 0.4, 0.21])

# Draw hexagonal mesh
linesHex = []

for face in dual_faces:
        hex_face = []
        for vi in face:
            hex_face.append(h_pts[vi])
        linesHex.append(vd.Line(hex_face, closed=True, lw=1.8, c='white'))
            
# Draw edges center mesh
center_edges = center.clone().wireframe().lw(1.5).flat().c('black').alpha(1)

# Draw edges Triangular mesh
edges = mesh.clone().lw(1.5).wireframe().flat().c('k').alpha(0.5)

# Draw axis lines
# Lines vd.Lines(ct- 5*nt, ct + 5*nt, c='red', lw=0.1)
clines = vd.Lines(ct[:5]- 5*nt[:5], ct[:5] + 5*nt[:5], c='tomato', lw=0.1)

vd.show(vd.Points(tv, c='red'), mesh, center, vd.Points(h_pts, c='red'), linesHex, edges, clines)
 

### Optimization Test

In [5]:
# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Init data files
file_pts = open(os.path.join( out_path, "points_"+str(k)+".dat"), "w")
ref = open(os.path.join( out_path, "init_directions_"+str(k)+".dat"), "w")


# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])


# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)
# Get inner vertices
inner_vertices = tmesh.inner_vertices()
# Get outer vertices
outer_vertices = tmesh.boundary_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual = tmesh.vertex_ring_faces_list()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])


# # Write data to file
for v  in tmesh.inner_vertices():
    file_pts.write(str(tmesh.vertices[v][0]) + " " + str(tmesh.vertices[v][1]) + " " + str(tmesh.vertices[v][2]) + "\n")
    ref.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")

file_pts.close()
ref.close()


# Optimization
# Init  X0 = [e_i | A| delta]
X = e_i.flatten() 

# Init constraints
linecong = LineCong()

linecong.initialize_constraint(X, len(tv), h_pts, dual, inner_vertices)
 
# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

for i in range(10):
    # Create file per iteration
    directions = open(os.path.join( out_path, "directions_"+str(k)+"_"+str(i)+".dat"), "w")

    # Compute J, r for each constraint
    linecong.compute(X,inner_vertices, dual)
    
    # Add constraints  
    opt.add_constraint(linecong)

    # Optimize
    opt.optimize("LM")

    e_i = opt.X.reshape(-1,3)   

    # Write data to file
    for v  in inner_vertices:   
        directions.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")
    directions.close()

# Print log
opt.print_log()
e_i = opt.X.reshape(-1,3)   
    
# Visualize line congruence
#congruence = vd.Lines(tv[inner_vertices[:5]]- 5*e_i[inner_vertices[:5]], tv[inner_vertices[:5]] + 5*e_i[inner_vertices[:5]], c=[0.8,0.5,0.5], lw=0.1)


Mesh Data Structure: |V| = 469, |F| = 858, |E| = 1326
 E: 78.03619781643368
 E: 4.548845207590189
 E: 0.8084802159231983
 E: 0.4928459320520198
 E: 0.4225571812416876
 E: 0.3915099908883715
 E: 0.3858314326786866
 E: 0.38041602184645906
 E: 0.3805343050506197
 E: 0.3793450973852111
      Energy
0  78.036198
1   4.548845
2   0.808480
3   0.492846
4   0.422557
5   0.391510
6   0.385831
7   0.380416
8   0.380534
9   0.379345


# Torsal test

In [12]:
def torsal_directions(v, f, e_i):
    num_faces = len(f)
    tors = np.zeros((2 * num_faces, 3))
    cos_tors = np.zeros(num_faces)
    for i in range(num_faces):
        # Extract vertex indices for the current face
        vertex_indices = f[i]

        # Calculate vectors e(u, v), e_u, e_v for the current face
        e_u, e_v = e_i[vertex_indices[1]] - e_i[vertex_indices[0]], \
                    e_i[vertex_indices[2]] - e_i[vertex_indices[0]]
        e = (e_i[vertex_indices[0]] + e_i[vertex_indices[1]] + e_i[vertex_indices[2]])/3

        # Calculate vectors a(u, v), a_u, a_v for the current face
        v_u, v_v = v[vertex_indices[1]] - v[vertex_indices[0]], \
                    v[vertex_indices[2]] - v[vertex_indices[0]]
        vv = (v[vertex_indices[0]] + v[vertex_indices[1]]+ v[vertex_indices[2]])/3

        # Calculate coefficients of the equation and solve it
        gamma_0 = np.linalg.det(np.array([e_u, v_u, e]))
        gamma_1 = np.linalg.det(np.array([e_u, v_v, e])) + np.linalg.det(np.array([e_v, v_u, e]))
        gamma_2 = np.linalg.det(np.array([e_v, v_v, e]))

        discr = gamma_1 * gamma_1 - 4.0 * gamma_0 * gamma_2
        # here should be exception about the negative discriminant

        # Negative discriminant
        if discr < 0 and abs(discr) > 1e-7:
            print("Discriminant is negative")
            tors[2*i] = np.array([0,0,0])
            tors[2 * i + 1] = np.array([0,0,0])
            cos_tors[i] = -1

        # Zero discriminant
        elif abs(discr) < 1e-7 :
            if discr < 0 : 
                soln = [(-gamma_1 ), (-gamma_1 )]
                sold = (2.0 * gamma_0)
            else:
                soln = [(-gamma_1 - np.sqrt(discr)), (-gamma_1 + np.sqrt(discr))]
                sold = (2.0 * gamma_0)
                     
            # Calculate torsal directions for the current face
            tors1 = soln[0] * v_u + sold * v_v
            tors1 = tors1 / np.linalg.norm(tors1)
            tors[2*i] = tors1
            tors2 = soln[1] * v_u + sold * v_v
            tors2 = tors2 / np.linalg.norm(tors2)
            tors[2 * i + 1] = tors2

            normal_plane_1 = np.cross(tors1, e)
            normal_plane_2 = np.cross(tors2, e)
            
            cos_tors[i] = abs(np.dot(normal_plane_1, normal_plane_2)) / (np.linalg.norm(normal_plane_1) * np.linalg.norm(normal_plane_2))

        # Positive discriminant
        else:            
            soln = [(-gamma_1 - np.sqrt(discr)), (-gamma_1 + np.sqrt(discr))]
            sold = (2.0 * gamma_0)
            
            # Calculate torsal directions for the current face
            tors1 = soln[0] * v_u + sold * v_v
            tors1 = tors1 / np.linalg.norm(tors1)
            tors[2*i] = tors1
            tors2 = soln[1] * v_u + sold * v_v
            tors2 = tors2 / np.linalg.norm(tors2)
            tors[2 * i + 1] = tors2

            normal_plane_1 = np.cross(tors1, e)
            normal_plane_2 = np.cross(tors2, e)
            
            cos_tors[i] = abs(np.dot(normal_plane_1, normal_plane_2)) / (np.linalg.norm(normal_plane_1) * np.linalg.norm(normal_plane_2))

    return tors, cos_tors
def torsal_dir_vec(tv, tf, e_i):
    
    # Get vertices
    vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]  

    # Get directions
    ei, ej, ek = e_i[tf[:,0]], e_i[tf[:,1]], e_i[tf[:,2]]

    # Compute edges
    eij = ej - ei
    eik = ek - ei

    # Compute vectors
    vij = vj - vi
    vik = vk - vi

    # Compute barycenter of directions
    ec = np.sum( e_i[tf], axis = 1)/3

    # Compute barycenter of vertices
    barycenters = (vi + vj+ vk)/3

    # Cross products
    vijXec = np.cross(vij, ec)
    vikXec = np.cross(vik, ec)

    # Compute coefficients
    g0 = np.sum(eij*vijXec, axis=1)
    g1 = np.sum(eij*vikXec, axis=1) + np.sum(eik*vijXec, axis=1)
    g2 = np.sum(eik*vikXec, axis=1)

    # Compute discriminant
    disc = g1**2 - 4*g0*g2
    

    # Get indices with negative discriminant, positive discriminant and small discriminant (i.e almost zero)
    negative_disc = np.where((disc < 0) & (np.abs(disc) > 1e-5))
    zero_disc = np.where((np.abs(disc) <= 1e-5))
    pos_disc = np.where((disc > 0) & (np.abs(disc) > 1e-5))

    # check disjoint sets
    assert len(np.intersect1d(negative_disc, zero_disc)) == 0
    assert len(np.intersect1d(negative_disc, pos_disc)) == 0
    assert len(np.intersect1d(zero_disc, pos_disc)) == 0

    # Init torsal directions
    t1 = np.zeros((len(disc), 3))
    t2 = np.zeros((len(disc), 3))

    # For discriminant near zero < 1e-5
    t1[zero_disc] = (-g1[zero_disc] + np.sqrt(abs(disc[zero_disc])))[:, None] * vij[zero_disc] + (2 * g0[zero_disc])[:, None] * vik[zero_disc]
    t2[zero_disc] = (-g1[zero_disc] - np.sqrt(abs(disc[zero_disc])))[:, None] * vij[zero_disc] + (2 * g0[zero_disc])[:, None] * vik[zero_disc]

    # For negative discriminant with absolute value > 1e-5
    t1[negative_disc] = np.zeros((len(negative_disc), 3))
    t2[negative_disc] = np.zeros((len(negative_disc), 3))

    # For positive discriminant > 1e-5
    t1[pos_disc] = (-g1[pos_disc] + np.sqrt(disc[pos_disc]))[:, None] * vij[pos_disc] + (2 * g0[pos_disc])[:, None] * vik[pos_disc]
    t2[pos_disc] = (-g1[pos_disc] - np.sqrt(disc[pos_disc]))[:, None] * vij[pos_disc] + (2 * g0[pos_disc])[:, None] * vik[pos_disc]

    # Normalize
    t1[zero_disc] /= np.linalg.norm(t1[zero_disc], axis=1)[:, None]
    t2[zero_disc] /= np.linalg.norm(t2[zero_disc], axis=1)[:, None]

    t1[pos_disc] /= np.linalg.norm(t1[pos_disc], axis=1)[:, None]
    t2[pos_disc] /= np.linalg.norm(t2[pos_disc], axis=1)[:, None]

    # Init cosines vectors
    cos_tors = np.zeros(len(disc))

    # Compute cross products
    t1Xec = np.cross(t1, ec)
    t2Xec = np.cross(t2, ec)

    # Get indices of nonzero vectors
    nonzeroInd = np.where(np.linalg.norm(t1Xec, axis=1) * np.linalg.norm(t2Xec, axis=1) >= 1e-5)

    # Compute cosines for nonzero vectors
    cos_tors[nonzeroInd] = np.sum( abs(t1Xec[nonzeroInd] * t2Xec[nonzeroInd]), axis=1) / (np.linalg.norm(t1Xec[nonzeroInd], axis=1) * np.linalg.norm(t2Xec[nonzeroInd], axis=1))

    # Compute cosines for zero vectors
    cos_tors[np.where(np.linalg.norm(t1Xec, axis=1) * np.linalg.norm(t2Xec, axis=1) < 1e-5)] = -1

    # print(f"f : 214 \n  disc : {disc[214]} \n ei : {ei[214]} \n ej : {ej[214]} \n ek : {ek[214]} \n eij : {eij[214]} \n eik : {eik[214]} \n vij : {vij[214]} \n vik : {vik[214]} \n g0 : {g0[214]} \n g1 : {g1[214]} \n g2 : {g2[214]} \n t1 : {t1[214]} \n t2 : {t2[214]} \n vijXec : {vijXec[214]} \n vikXec : {vikXec[214]} \n ec : {ec[214]} \n barycenters : {barycenters[214]} \n cos_tors : {cos_tors[214]}")

    return  barycenters, t1, t2, cos_tors

In [8]:
# Check time
start = time.time()
bar, t1, t2, cos_tors = torsal_dir_vec(tv, tf, e_i)
end = time.time()
print(f"t1 : \n{t1[:5]} \n t2 :\n {t2[:5]} \n cos_tors :\n {cos_tors[:5]}")
print("Time for torsal directions Vec: ", end-start)

t1 : 
[[-0.9627944  -0.18558465 -0.19643135]
 [-0.97948834 -0.17999254 -0.09058299]
 [ 0.03650969  0.9946874   0.0962498 ]
 [ 0.88488844  0.46577062 -0.00549409]
 [ 0.99981114  0.01833656  0.00643836]] 
 t2 :
 [[-0.38973324  0.91863383 -0.06496073]
 [-0.7544647   0.58152454  0.30432257]
 [-0.92537749  0.02619593 -0.37814054]
 [ 0.20356354  0.9767577   0.06712885]
 [ 0.5870602   0.80657291  0.0692853 ]] 
 cos_tors :
 [0.62569669 0.84748461 0.07732948 0.62432098 0.58821174]
Time for torsal directions Vec:  0.005192279815673828


In [ ]:
start = time.time()
tors, cos_tors = torsal_directions(tv, tf, e_i)
end = time.time()
# Get even indices
tors1 = tors[::2]
# Get odd indices
tors2 = tors[1::2]
print(f"t1 : \n{t1[:5]} \n t2 :\n {t2[:5]} \n cos_tors :\n {cos_tors[:5]}")
print("Time for torsal directions: ", end-start)

In [30]:


tpos = open(os.path.join( out_path, "torsal_pos_"+str(k)+".dat"), "w")
tdir = open(os.path.join( out_path, "torsal_dir_nor_"+str(k)+".dat"), "w")
bar, t1, t2, cos_tors = torsal_dir_vec(tv, tf, e_i)

tors, cos_tors = torsal_directions(tv, tf, e_i)
# Get even indices
tors1 = tors[::2]
# Get odd indices
tors2 = tors[1::2]

t_mesh = Mesh()
t_mesh.make_mesh(tv, tf)

inner_faces = t_mesh.inner_faces()

for ii in range(len(inner_faces)):
    i = inner_faces[ii]
    tpos.write(str(bar[i][0]) + " " + str(bar[i][1]) + " " + str(bar[i][2]) + "\n")
    tdir.write(str(t1[i][0]) + " " + str(t1[i][1]) + " " + str(t1[i][2]) + " " + str(t2[i][0]) + " " + str(t2[i][1]) + " " + str(t2[i][2]) + "\n")

tpos.close()
tdir.close()


badf = np.array([214,649,78,226,653])

print(f"e : \n{e_i[tf[badf]]} \n t1 :\n {t1[badf]} \n t2 :\n {t2[badf]} \n cos_tors :\n {cos_tors[badf]}")

#igl.write_triangle_mesh(os.path.join( out_path, "tri_"+str(k)+".obj"), tv, tf)

f : 214 
  disc : -5.873123945025552e-07 
 ei : [-0.2793207  -0.26385476  0.92265852] 
 ej : [-0.31704103 -0.25471345  0.91297133] 
 ek : [-0.30817463 -0.29257547  0.90466743] 
 eij : [-0.03772032  0.00914131 -0.00968719] 
 eik : [-0.02885393 -0.02872071 -0.01799109] 
 vij : [ 0.163697 -0.051988  0.005017] 
 vik : [0.142775 0.121967 0.009283] 
 g0 : 0.0009399914046376113 
 g1 : 0.0012502315294135322 
 g2 : 0.0005719178018631141 
 t1 : [0.5960329  0.80156394 0.0473291 ] 
 t2 : [-0.1815324   0.98314813  0.02158119] 
 vijXec : [-0.04613102 -0.15103883 -0.05993561] 
 vikXec : [ 0.11391856 -0.13321425 -0.00182915] 
 ec : [-0.30151212 -0.27038123  0.91343242] 
 barycenters : [ 1.78239833  1.67626833 -1.11108133] 
 cos_tors : 0.8620652800296591
Discriminant is negative
Mesh Data Structure: |V| = 469, |F| = 858, |E| = 1326
e : 
[[[-0.2793207  -0.26385476  0.92265852]
  [-0.31704103 -0.25471345  0.91297133]
  [-0.30817463 -0.29257547  0.90466743]]

 [[-0.23104326 -0.32883748  0.91512483]
  [-0.

In [ ]:
# Initialize polyscope
ps.init()


# Register a Mesh
v_tmesh = ps.register_surface_mesh("Mesh", tv, tf, smooth_shade=True)
ps.register_surface_mesh("Sphere Centers Mesh", h_pts, dual_faces, smooth_shade=True, edge_color=[0.8,0.5,0.5], edge_width=0.1)

# Draw circumcenter axis
# Register a point cloud
#v_cc = ps.register_point_cloud("Circumcenter ", tv, radius=0.01)

#print("ei :", e_i.shape)
#print("ct :", ct.shape)
# basic visualization
v_tmesh.add_vector_quantity("Axis", e_i, defined_on='vertices', enabled=True, radius=0.0001, length=5, color=(0.8, 0.0, 0.2))


# View the point cloud and mesh we just registered in the 3D UI
ps.show()

# Optimization Hyperbolicity

## Test 1 triangle

In [2]:
import igl
#import meshplot as mp
import vedo as vd
import polyscope as ps
import numpy as np
import os 
import time 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.HyperbolicLC import HyperbolicLC
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

# tv test
tv = np.array([
    [0,0,1]  , # v0
    [1,0,0]  , # v1
    [0,1,0]  , # v2
    [0,0,-1] , # v3
    [-1,0,0] , # v4
    [0,-1,0] , # v5
    [-1,-1,0] # v6
    ])

# tf test
tf = np.array([
    [0,1,2],
    [0,2,3],
    [0,3,4],
    [0,4,5],
    [0,5,6]
    ])

# e_i test
e_i = np.array([
    [0,0,0.2],
    [0,0.4,0.4],
    [0.5,0.5,0.5],
    [0.5,0.5,-0.5],
    [0.5,-0.5,0.5],
    [0.5,-0.5,-0.5],
    [-0.5,0.0, 0.0]
    ])


# h pts test
h_pts = np.mean(tv[tf], axis=1) + np.cross(tv[tf[:,1]] - tv[tf[:,0]], tv[tf[:,2]] - tv[tf[:,0]]) / (2 * np.linalg.norm(np.cross(tv[tf[:,1]] - tv[tf[:,0]], tv[tf[:,2]] - tv[tf[:,0]]), axis=1))[:, None]


t_mesh = Mesh()
t_mesh.make_mesh(tv, tf)

dual = t_mesh.vertex_ring_faces_list()
inner_vertices = t_mesh.inner_vertices()

e_i = e_i / np.linalg.norm(e_i, axis=1)[:, None]

# Compute the directions at the barycenters
ec = np.sum( e_i[tf], axis = 1) / 3

# # Compute the edge vectors per each face
vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]


# # Compute the edge vectors per each face
vij = vj - vi
vik = vk - vi

# Set up X 
#X[:3*self.nV] = V.flatten()

eij = e_i[tf[:,1]] - e_i[tf[:,0]]
eik = e_i[tf[:,2]] - e_i[tf[:,0]]

# A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
# A = det1 +  det2
eikXec = np.cross(eik, ec)
vikXec = np.cross(vik, ec)

det1 = np.sum(vij*eikXec, axis=1)
det2 = np.sum(eij*vikXec, axis=1)

# b = [eij, eik, ec]  c = [vij, vik, ec]

b = np.sum(eij*eikXec, axis=1)
c = np.sum(vij*vikXec, axis=1)

A = det1 + det2 

# variables = [e_i | A| delta]; e_i direction per vertex in T and A is one per each triangle in T, same for delta 
X = np.zeros(3*len(tv) + 2*len(tf) )

# Optimization
X[:3*len(tv)] = e_i.flatten()



# Init constraints ---------------------------------------------
# # Init LineCong
linecong = LineCong()
# X variables, e_i dim, pts in hexagonal mesh, faces in hexagonal mesh, inner vertices indices.
linecong.initialize_constraint(X, len(tv), h_pts, dual, inner_vertices)

# # Init Hyperbolic
hyp = HyperbolicLC()
w = 1
X = hyp.initialize_constraint(X, tv, tf, e_i, 4, w)


# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

# Compute J, r for each constraint
#linecong.compute(inner_vertices, dual, X)

# Add constraints  
#opt.add_constraint(linecong)
time_opt = []

for i in range(20):

    # Take time
    start = time.time()
    hyp.compute(X, tf) 
    #linecong.compute(X, inner_vertices, dual)

    opt.add_constraint(hyp)
    #opt.add_constraint(linecong)

    # Optimize
    opt.optimize("LM")
    end = time.time()

    #print(f"delta : {opt.X[-1]}")

    time_opt.append(end - start)


#opt.print_log()

#print(f"time:\n {time_opt}")

e_i = opt.X[:3*len(tv)].reshape(-1,3)

print(np.linalg.norm(e_i, axis=1))

eij = e_i[tf[:,1]] - e_i[tf[:,0]]
eik = e_i[tf[:,2]] - e_i[tf[:,0]]


ec = np.sum( e_i[tf], axis = 1) / 3
# A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
# A = det1 +  det2
eikXec = np.cross(eik, ec)
vikXec = np.cross(vik, ec)

det1 = np.sum(vij*eikXec, axis=1)
det2 = np.sum(eij*vikXec, axis=1)

# [vij, vik, ec] = det1 
vikXec = np.cross(vik, ec)
detA = np.sum(vij*vikXec, axis=1)

# [eij, eik, ec] = det2
eikXec = np.cross(eik, ec)
detC = np.sum(eij*eikXec, axis=1)
# b = [eij, eik, ec]  c = [vij, vik, ec]

A = det1 + det2 

Ac = opt.X[3*len(tv):3*len(tv)+len(tf)]

print(f"Disc f: {Ac**2 - 4*detA*detC} \t delta: { opt.X[3*len(tv) + len(tf):]} \tenergy: {(Ac**2 - 4*detA*detC - opt.X[3*len(tv) + len(tf):]**2)**2}")

print(f"Diff A {abs(Ac - A)}")

Mesh Data Structure: |V| = 7, |F| = 5, |E| = 11
 E: 1173.0156201595648
 E: 71636.02852981041
 E: 8972.629949425555
 E: 829.7576944449751
 E: 50.08084593164021
 E: 2.4527869783699243
 E: 0.1049777638852806
 E: 0.004268469671451031
 E: 0.0001713507913714364
 E: 6.860042416955949e-06
 E: 2.744922938558556e-07
 E: 1.0982113842165888e-08
 E: 4.3937184643760367e-10
 E: 1.757829085409877e-11
 E: 7.032676374462893e-13
 E: 2.8136154220880402e-14
 E: 1.1256625917902005e-15
 E: 4.5035499751803513e-17
 E: 1.8017949968017823e-18
 E: 7.209755481797998e-20
[3.13064081 2.70211335 3.26050199 2.74996373 6.04061451 3.65838457
 3.68173939]
Disc f: [16. 16. 16. 16. 16.] 	 delta: [4. 4. 4. 4. 4.] 	energy: [3.92996619e-24 9.91668714e-23 7.43299783e-22 1.89405503e-21
 1.39206068e-22]
Diff A [ 2.78027786  5.80079956 14.53705357 14.44584076 14.04189853]


## Test data

### Load Data

In [1]:
import igl
#import meshplot as mp
import vedo as vd
import polyscope as ps
import numpy as np
import os 
import time 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.HyperbolicLC import HyperbolicLC
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

#vd.settings.default_backend = 'k3d'
# # Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Init data files
file_pts = open(os.path.join( out_path, "points_"+str(k)+".dat"), "w")
ref = open(os.path.join( out_path, "init_directions_"+str(k)+".dat"), "w")

# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 

# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)

# Get inner vertices
inner_vertices = tmesh.inner_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual = tmesh.vertex_ring_faces_list()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

Mesh Data Structure: |V| = 469, |F| = 858, |E| = 1326


In [3]:
# Compute the norms of ec
#nc = np.linalg.norm(ec, axis=1)
def compute_disc(tv, tf, e_i):

    # # Compute the edge vectors per each face
    vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]

    # # Compute the edge vectors per each face
    vij = vj - vi
    vik = vk - vi

    # Set up X 
    eij = e_i[tf[:,1]] - e_i[tf[:,0]]
    eik = e_i[tf[:,2]] - e_i[tf[:,0]]

    ec = np.sum( e_i[tf], axis = 1) / 3

    # A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
    # A = gamma11 +  gamma12
    eikXec = np.cross(eik, ec)
    vikXec = np.cross(vik, ec)

    det1 = np.sum(vij*eikXec, axis=1)
    det2 = np.sum(eij*vikXec, axis=1)

    # b = [eij, eik, ec]  c = [vij, vik, ec]

    gamma0 = np.sum(eij*eikXec, axis=1)
    gamma2 = np.sum(vij*vikXec, axis=1)

    A = det1 + det2 

    return A, A**2 - 4*gamma0*gamma2

A, disc = compute_disc(tv, tf, e_i)

# variables = [e_i | A| delta]; e_i direction per vertex in T and A is one per each triangle in T, same for delta 
X = np.zeros(3*len(tv) + 2*len(tf) )

# Optimization
X[:3*len(tv)] = e_i.flatten()


# Init constraints ---------------------------------------------
# # Init LineCong
linecong = LineCong()
# X variables, e_i dim, pts in hexagonal mesh, faces in hexagonal mesh, inner vertices indices.
linecong.initialize_constraint(X, len(tv), h_pts, dual, inner_vertices, 1)

np.random.seed(0)
# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

# # Init Hyperbolic
hyp = HyperbolicLC()
X = hyp.initialize_constraint(X, tv, tf, e_i, 1, 1)

# Add constraints  
#opt.add_constraint(linecong)
time_opt = []

for i in range(40):

    # Take time
    start = time.time()
    hyp.compute(X, tf)
    #linecong.compute(X, inner_vertices, dual)


    opt.add_constraint(hyp)
    #opt.add_constraint(linecong)

    # Optimize
    opt.optimize("LM")
    end = time.time()

    X = opt.X

e_i = X[:3*len(tv)].reshape(-1,3)
A, disc = compute_disc(tv, tf, e_i)

delta = X[3*len(tv) + len(tf):]
print(disc[:5])
print(delta[:5]**2)
#print(f"Av time:\n {np.mean(time_opt)}")

 E: 857.8286590197176
 E: 111.16442044279222
 E: 14.403095105608998


In [5]:
e_i = opt.X[:3*len(tv)].reshape(-1,3)

#print(np.linalg.norm(e_i, axis=1))

eij = e_i[tf[:,1]] - e_i[tf[:,0]]
eik = e_i[tf[:,2]] - e_i[tf[:,0]]


ec = np.sum( e_i[tf], axis = 1) / 3
# A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
# A = det1 +  det2
eikXec = np.cross(eik, ec)
vikXec = np.cross(vik, ec)
eijXec = np.cross(eij, ec)

det1 = np.sum(vij*eikXec, axis=1)
det2 = np.sum(eij*vikXec, axis=1)

# [vij, vik, ec] = det1 
vikXec = np.cross(vik, ec)
detA = np.sum(vij*vikXec, axis=1)

# [eij, eik, ec] = det2
eikXec = np.cross(eik, ec)
detC = np.sum(eij*eikXec, axis=1)
# b = [eij, eik, ec]  c = [vij, vik, ec]

A = det1 + det2 
A1 = det1 - np.sum(vik*eijXec, axis=1)

Ac = opt.X[3*len(tv):3*len(tv)+len(tf)]

neg = np.where(A**2 - 4*detA*detC  < 0)

# print meann time 
print(f"Av time:\n {np.mean(time_opt)}")
delta = opt.X[3*len(tv) + len(tf):]

print(f"sqr Disc f: {np.sqrt( abs ( (A**2 - 4*detA*detC)[neg[0]]))}  \n num neg: {len(neg[0])}\n delta: { delta[neg[0]]**2} \n disc :{(A**2 - 4*detA*detC)[neg[0]] }")
print(f"energy hyp [neg]: {((Ac**2 - 4*detA*detC)[neg[0]] - delta[neg[0]]**2)} ")
print(f"First 5: disc {(Ac**2 - 4*detA*detC)[:5]}")
#print(f"energy A [:5]: {(Ac - det1 - det2)**2}")

 

Av time:
 nan
sqr Disc f: [2.28721233e-04 7.14869068e-05 7.77451280e-04 3.48570791e-04
 2.24137361e-04 7.67060034e-04 6.18219620e-04 9.95752527e-04
 7.13690606e-04 1.11553581e-03 1.27659907e-04 2.58645718e-04
 2.74777344e-04 1.19918827e-03 8.02359582e-04 7.92996637e-04
 2.59058331e-04 3.74343533e-03 1.53588667e-03 2.62286174e-04
 9.31041583e-04 2.97120308e-04 4.94358809e-04 7.23334967e-04
 1.74359855e-04]  
 num neg: 25
 delta: [3.50165481e-03 5.13549817e-05 2.46412031e-03 3.68499720e-05
 2.39950866e-06 1.34659097e-06 2.78416165e-03 1.72751719e-03
 2.14132672e-03 3.52377665e-03 2.76429479e-03 6.57728849e-04
 2.84256614e-03 7.37178324e-04 3.97554324e-05 2.00238208e-03
 4.90909418e-06 3.28623793e-04 3.36194160e-03 1.00551115e-03
 5.18496689e-04 1.66938778e-03 5.20797837e-04 1.48553098e-03
 6.13970406e-04] 
 disc :[-5.23134024e-08 -5.11037784e-09 -6.04430493e-07 -1.21501596e-07
 -5.02375566e-08 -5.88381095e-07 -3.82195499e-07 -9.91523094e-07
 -5.09354281e-07 -1.24442015e-06 -1.62970518e-0

In [13]:
tpos = open(os.path.join( out_path, "torsal_pos_"+str(k)+".dat"), "w")
tdir = open(os.path.join( out_path, "torsal_dir"+str(k)+".dat"), "w")
bar, t1, t2, cos_tors = torsal_dir_vec(tv, tf, e_i)

t_mesh = Mesh()
t_mesh.make_mesh(tv, tf)

inner_faces = t_mesh.inner_faces()

for ii in range(len(inner_faces)):
    i = inner_faces[ii]
    tpos.write(str(bar[i][0]) + " " + str(bar[i][1]) + " " + str(bar[i][2]) + "\n")
    tdir.write(str(t1[i][0]) + " " + str(t1[i][1]) + " " + str(t1[i][2]) + " " + str(t2[i][0]) + " " + str(t2[i][1]) + " " + str(t2[i][2]) + "\n")

tpos.close()
tdir.close()




Mesh Data Structure: |V| = 469, |F| = 858, |E| = 1326


### Visualization

In [14]:
e_i = opt.X[:3*len(tv)].reshape(-1,3)

bar, t1, t2, cos_tors = torsal_dir_vec(tv, tf, e_i)

tors, cos_tors1 = torsal_directions(tv, tf, e_i)

tt1 = tors[::2]
tt2 = tors[1::2]

print(cos_tors[np.where(cos_tors == -1)[0]])

# # # Initialize polyscope
ps.init()

# Register a Mesh
v_tmesh = ps.register_surface_mesh("Mesh", tv, tf, smooth_shade=True)
#ps.register_surface_mesh("Sphere Centers Mesh", h_pts, dual, smooth_shade=True, edge_color=[0.8,0.5,0.5], edge_width=0.1)

# Draw baricenters
v_cc = ps.register_point_cloud("barycenters ", bar, radius=0.001)

# Add torsal directions
v_tmesh.add_vector_quantity("t1", t1, defined_on='faces', enabled=True, radius=0.001, length=0.01, color=(0.8, 0.0, 0.5))
v_tmesh.add_vector_quantity("-t1", -t1, defined_on='faces', enabled=True, radius=0.001, length=0.01, color=(0.8, 0.0, 0.5))
v_tmesh.add_vector_quantity("t2", t2, defined_on='faces', enabled=True, radius=0.001, length=0.01, color=(0.1, 0.1, 0.1))
v_tmesh.add_vector_quantity("-t2", -t2, defined_on='faces', enabled=True, radius=0.001, length=0.01, color=(0.1, 0.1, 0.1))
#v_tmesh.add_vector_quantity("t2", tt2, defined_on='faces', enabled=True, radius=0.005, length=0.01, color=(0.2, 0.0, 0.8))
#v_tmesh.add_vector_quantity("-t2", -tt2, defined_on='faces', enabled=True, radius=0.005, length=0.01, color=(0.2, 0.0, 0.8))

# View the point cloud and mesh we just registered in the 3D UI
ps.show()

f : 214 
  disc : -5.883822780751981e-07 
 ei : [-0.27943394 -0.26396173  0.92303249] 
 ej : [-0.31717957 -0.25482476  0.91337021] 
 ek : [-0.30831394 -0.29270772  0.90507629] 
 eij : [-0.03774563  0.00913697 -0.00966227] 
 eik : [-0.02887999 -0.02874599 -0.0179562 ] 
 vij : [ 0.163697 -0.051988  0.005017] 
 vik : [0.142775 0.121967 0.009283] 
 g0 : 0.0009407276128328533 
 g1 : 0.001251352945089118 
 g2 : 0.0005725000632146782 
 t1 : [0.59601456 0.80157759 0.04732877] 
 t2 : [-0.18163743  0.98312883  0.02157643] 
 vijXec : [-0.04615091 -0.15110397 -0.05996151] 
 vikXec : [ 0.11396769 -0.1332717  -0.00182993] 
 ec : [-0.30164248 -0.27049807  0.91382633] 
 barycenters : [ 1.78239833  1.67626833 -1.11108133] 
 cos_tors : 0.862117781325683
Discriminant is negative
Discriminant is negative
Discriminant is negative
[]


## Minimize Library test

In [ ]:
from scipy import optimize


def energy(x0, ei_dim, cicj, dual_faces):

    # energy \sum_{f \in F} \sum_{cj,ci \in E(f)} || e_f (cj - ci)  ||^2
    cf = dual_faces
    
    # Init energy value
    total_energy = 0

    # Get directions
    ei = x0.reshape(ei_dim, 3)

    # Loop over faces
    for f in range(len(cf)):
        # Define Jacobian
        diff = np.dot(cicj[f], ei[f])

        # Define residual
        total_energy += np.sum( diff**2, axis=0)
    
    diff2 = np.sum(ei * ei, axis=1) - 1

    total_energy += np.sum(diff2**2, axis=0)

    return total_energy

# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])


# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create test mesh
test_mesh =  vd.Mesh((tv, tf), alpha = 0.8)

# Draw edges Triangular mesh
edges = test_mesh.clone().lw(1.5).wireframe().flat().c('k').alpha(0.5)

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)
# Get inner vertices
inner_vertices = tmesh.inner_vertices()
# Get outer vertices
outer_vertices = tmesh.boundary_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual_faces = tmesh.dual_top()

# Dual topology without outer faces
#dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

# Compute cij
cij = []
# Loop over faces
for f in range(len(dual_faces)):
    # Get face
    face = dual_faces[f]

    # Get vertices
    v0 = h_pts[face]
    v1 = np.roll(h_pts[face], -1, axis=0)
 
    cicj = (v1 - v0)/np.linalg.norm(v1 - v0, axis=1)[:, None]
    # Define direction
    cij.append(cicj)

dim = len(inner_vertices)


opt = optimize.minimize(energy, e_i[inner_vertices].flatten(), args=(dim, cij, dual_faces), options={'disp': True})


In [ ]:
e_i = opt.x.reshape(-1,3)

minfile = open(os.path.join( out_path, "min_directions_"+str(k)+".dat"), "w")

for v  in range(len(tmesh.inner_vertices())):   
    minfile.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")
minfile.close()



### Test Circumcircle

In [ ]:

# Get a list of random points
#p = np.random.rand(10,3)

p1 = np.random.rand(10,3)
p2 = np.random.rand(10,3)
p3 = np.random.rand(10,3)

c, rad, n = circle_3pts(p1,p2,p3)
print(f"centers: {c}\t rad: {rad}\t n: {n}")
circles = []
for ci in range(len(c)):
    print(f"centers: {c[ci]}\t rad: {rad[ci]}\t n: {n[ci]}")
    if ci%2 == 0:
        circles.append( vd.Circle(c[ci], r=rad[ci], c='red', alpha = 0.5).orientation(n[ci])) 
    else:
        circles.append( vd.Circle(c[ci], r=rad[ci], c="green", alpha = 0.5).orientation(n[ci])) 
pts1 = vd.Points(np.array( np.vstack((p1, p2, p3))), r = 10, c = 'black')
#pts2 = vd.Points(np.array([p1[1], p2[1], p3[1]]), r = 10, c = 'green')

vd.show(pts1,*circles)


## Sample Planarity Opt

In [ ]:
# Create four quads in 3D that merge in a vertex
v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
fcs = np.array([[0, 1, 6, 3], [2, 0, 3, 5], [7, 4, 0, 2], [4, 8, 1, 0]])

# Make mesh
m = Mesh()
m.make_mesh(v, fcs)

mesh1 = vd.Mesh((v, fcs), alpha = 0.8, c='r')

X = np.zeros((3*m.V+3*m.F), dtype=np.float64)

X[:3*m.V] = m.vertices.flatten()

# Init constraints
planarity = Planarity()
planarity.initialize_constraint(m, X)


# init optimizer
opt = Optimizer()

opt.initialize_optimizer(X)

for i in range(10):

    # Compute J, r for each constraint
    planarity.compute(m, X)
    
    # Add constraints  
    opt.add_constraint(planarity)
    opt.optimize("LM")
 

mesh2 = vd.Mesh((opt.X[:3*m.V].reshape(m.V,3),  fcs), alpha = 0.9, c='b')

print(opt.energy)

vd.show(mesh1, mesh2)



In [ ]:

v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
fcs = np.array([[0, 1, 6, 3], [2, 0, 3, 5], [7, 4, 0, 2], [4, 8, 1, 0]])

nv = np.array([[-0.08452795, -0.5184518 ,  0.1204968 ],
       [ 0.09325778, -0.5417252 , -0.14323919],
       [-0.39126414,  0.38199818,  0.21744937],
       [ 0.05301311,  0.4957555 , -0.02630325],
       [-0.04160952, -0.68063384,  0.11461049],
       [-0.64098644,  0.96492183,  0.3065665 ],
       [ 0.29607683,  0.33823755, -0.46029344],
       [-0.24634174, -0.29367885,  0.3622692 ],
       [ 0.5331171 , -1.4693466 , -0.43155265]]
       )


mesh1 = vd.Mesh((v, fcs), alpha = 0.8, c='r')
mesh2 = vd.Mesh((nv, fcs), alpha = 0.8, c='b')


vd.show(mesh1, mesh2)

In [ ]:
v, f = igl.read_triangle_mesh("models/catenoid_def_1.obj")
mesh = om.read_trimesh("models/catenoid_def_1.obj")
mp.plot(v, f, c=v[:, 0])

In [ ]:
def iso_stereo(n, h):
    # Isotropic model stereographic projection
    # n: normal vector
    # h: distance center to plane
    # return point in I3

    # Concatenate n[1:2] and h
    return np.vstack((n[1:2],v))/(1+ n[2])

In [ ]:
a = np.array([[1, 2, 3],[5,4,2] , [9,7,2]])
b = np.array([19,22,89])
# Add b as a column to a
c = np.column_stack((a,b))
print(a[:, 0:2])

In [ ]:
# Get face normals
nf = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
# Get indices v1 of each face
iv1 = f[:, 0]
# Get vertices of idv1
v1 = v[iv1, :]
# Compute dot product of each face normal with v1
h = np.sum(nf * v1, axis=1)

# Get Isotropic point per face
nv = np.column_stack((nf[:, 0:2], h)) / (1 + nf[:, 2, None])

nf = np.empty((0, 3), dtype=np.int32)

for fi in range(len(f)):
    aux = np.empty((0, ), dtype=np.int32)
    for adf in mesh.ff(mesh.face_handle(fi)):
        aux = np.hstack((aux, [adf.idx()]))
    if len(aux) == 3:
        nf = np.vstack((nf, aux))

mp.plot(nv, nf[:2])

In [ ]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix

v, f = igl.read_triangle_mesh("models/Hall.obj")

# Draw the mesh
n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
c = np.linalg.norm(n, axis=1)
p = mp.plot(v, f, c, return_plot=True) # plot

# Compute Cuvature properties
v1, v2, k1, k2 = igl.principal_curvature(v, f)



# Compute focal surfaces
rho1 = 100
rho2 = 1/k2

step = 0.001

lamb = 0.1 * np.ones(v.shape[0], dtype=np.float64)

J = np.eye(v.shape[0], dtype=np.float64) 

# Precalculate intermediate states
vs = [v]
cs = [c]
for i in range(30):
    # Compute the gradient of the focal surface
    H = csr_matrix(J.T@J)
    b = - J.T@(lamb - rho1)
    delta = spsolve(H, b)
    lamb = lamb + step * delta
    # Update the mesh
    v = v + lamb[:,None]*n

    # Recompute the normals
    n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
    c = np.linalg.norm(n, axis=1)
    # Add to the list
    # p.update_object(vertices=v, colors=c)
    vs.append(v)
    cs.append(c)
    

# Add interactive visulization
@mp.interact(level=(0, 9))
def mcf(level=0):
    p.update_object(vertices=vs[level], colors=cs[level])
    vs.append(v)
    cs.append(c)
p